# Paillier Crypto System

1999 von Pascal Paillier vorgestellt in seinem Paper:

[Public-Key Cryptosystems Based on Composite
Degree Residuosity Classes](Seminararbeit/Literatur/paillier_pubKeyCryptosystem.pdf)

Es ist ein __asymmetrisches Verschlüsselungsverfahren__.

Daher gibt es sowohl einen __public Key__ wie auch einen __private Key__.

Mithilfe des __public Key__ können andere Daten verschlüsseln, die nur der Besitzer des __private Keys__ entschlüsseln kann.


## Primzahlen
Für die Erzeugung der beiden Keys benötigen wir:
- zwei ausreichend große Primzahlen
  - mit selber Bit-Länge
  - hier __STARK__ begrenzte Bit-Länge

*Im realen Gebrauch sollten die Primzahlen jeweils mindestens 1024 bits haben, besser 2048 bits oder mehr. Die Zerlegung in Primfaktoren wird erschwert, allerdings wird auch der Rechenaufwand erhöht.* 

In [1]:
from sympy import randprime

def create_primes(bits):

    # declare bit-length of primes
    # 4 bit primes, between 8 and 16: 11, 13
    bit_length = bits

    # Generate first bit-length prime number
    p = randprime(2**(bit_length-1), 2**bit_length - 1)

    # Generate second bit-length prime number different from the first one
    q = randprime(2**(bit_length-1), 2**bit_length - 1)
    while q == p:
        q = randprime(2**(bit_length-1), 2**bit_length - 1)

    bit_length_p = p.bit_length()
    bit_length_q = q.bit_length()

    print(f"Bit length of {p} is: {bit_length_p} bits")
    print(f"Bit length of {q} is: {bit_length_q} bits")
    return p, q

p, q = create_primes(8)

Bit length of 197 is: 8 bits
Bit length of 251 is: 8 bits


### größter gemeinsamer Teiler

Die gewählten Primzahlen müssen: 

$ggT(pq,(p-1) * (q-1)) = 1$

erfüllen.

Wobei, $n = p \cdot q$ ist und

$\varphi = (p-1) \cdot (q-1)$ die Eulersche Phi Funktion ist.

In [2]:
n = (p * q)
phi = (p - 1) * (q - 1)

def ggT(a, b):
    while b != 0:
        a, b = b, a % b
    return a

In [3]:
result_ggT = ggT(n, phi)
print("ggT = ", result_ggT)
assert result_ggT == 1

ggT =  1


## Public Key

Der __public Key__ besteht aus: $n, g$.

$n=(p\cdot q)$

$g:$ zufällig gewähltes $g$ aus $({\mathbb {Z}}/n^{2}{\mathbb {Z}})^{*}$, so dass $n$ ($n=p \cdot q$) die Ordnung von $g$ teilt.

In [4]:
import random
from sympy import isprime

# num has to be prime and ggT(num, n^2) == 1
def multi_group_g(n):
    group_g = []
    for num in range(1, n*n):
        if isprime(num) and ggT(num, n*n) == 1:
            group_g.append(num)
    return group_g

# random choice of g out of group_g
group_g = multi_group_g(n)
g = random.choice(group_g)

print("Multiplikative Gruppe (Z/n^2Z)* für n =", n, "enthält", len(group_g), "Elemente.\nRandom int g: ", g)

# final public Key
pubKey = n, g
print("Public Key: ", pubKey)

Multiplikative Gruppe (Z/n^2Z)* für n = 49447 enthält 118901097 Elemente.
Random int g:  1147970869
Public Key:  (49447, 1147970869)


## Private Key

Der __private Key__ besteht aus $\lambda$ und $\mu$.

### $\lambda$

$\lambda$ ist das kleinste gemeinsame Vielfache (lowest common multiple)

$\lambda = kgV(p - 1, q - 1)$

$kgV = \frac{|a \cdot b|}{ggT(a,b)}$


In [5]:
# kgV
def lcm(a, b):
    return abs(a * b) // ggT(a, b)

# lambda
def calc_lmbda(p, q):
    lmbda = lcm(p-1, q-1)
    return lmbda

lmbda = calc_lmbda(p, q)
print("Lambda: ", lmbda)

Lambda:  24500


### $\mu$

$L(x)= \frac{(x-1)}{n}$

$\mu = (L(g^\lambda \mod n^2))^{-1} \mod n$

In [6]:
# L(x)
def L(x, n):
    return ((x - 1) // n)

# Mu
def calc_mu(g, lmbda, n):
    x = pow(g, lmbda, n**2)
    temp = L(x, n)
    mu = int(pow(temp, -1, n))
    return mu

mu = calc_mu(g, lmbda, n)

# final private Key
privKey = lmbda, mu
print("Private Key: ", privKey)

Private Key:  (24500, 30061)


## Verschlüsselungsprozess

Wir setzen $m =$ Klartext mit $m \in (\mathbb{Z} / n \mathbb{Z})$, d.h. $m<n$.

Damit ist der Ciphertext $c$:

$c = g^m \cdot r^n \mod n^2$

$r:$ zufällig gewähltes $r \in (\mathbb{Z} / n \mathbb{Z})^*$

In [7]:
# num has to be prime and ggT(num, n) == 1
def multi_group_r(n):
    group_r = []
    for num in range(1, n):
        if isprime(num) and ggT(num, n) == 1:
            group_r.append(num)
    return group_r

# random choice of r out of group_r
def choose_r(n):
    group_r = multi_group_r(n)
    r = random.choice(group_r)
    print("Multiplikative Gruppe (Z/nZ)* für n =", n, "enthält", len(group_r), "Elemente.\nRandom int r: ", r)
    return r

r = choose_r(n)

Multiplikative Gruppe (Z/nZ)* für n = 49447 enthält 5078 Elemente.
Random int r:  25423


#### Verschlüsselung

Für die __Verschlüsselung__ benötigen wir den __Public Key__ , $r$ und einen Klartext.

In [8]:
def encryption(g, m, r, n):
    c =  pow(g, m) * pow(r, n) % n**2
    return c

In [9]:
# Example Numbers
num1 = 2
num2 = 7

c1 = encryption(g, num1, r, n)
c2 = encryption(g,num2,r,n)

print("Klartext num1: ", num1, "\nCiphertext c1: ", c1 , "\n")
print("Klartext num2: ", num2, "\nCiphertext c2: ", c2)

Klartext num1:  2 
Ciphertext c1:  2196859067 

Klartext num2:  7 
Ciphertext c2:  551182097


## Entschlüsselungsprozess

$m = L(c^{\lambda} \mod n^2) \cdot \mu \mod n$

Hierbei wird der __Private Key__ verwendet.

$L(x)$ ist die oben definierte Funktion.

In [10]:
def decryption(c, lmbda, n, mu):
    temp_Lx = pow(c, lmbda, n**2)
    k =  (L(temp_Lx, n) * mu) % n
    return k

In [11]:
k1 = decryption(c1, lmbda, n, mu)
k2 = decryption(c2, lmbda, n, mu)
print("entschlüsselter Klartext num1: ", k1)
print("entschlüsselter Klartext num2: ", k2)

entschlüsselter Klartext num1:  2
entschlüsselter Klartext num2:  7


# Die Addition als Homomorphe Eigenschaft

Durch Multiplikation zweier Ciphertexte, werden die Klartexte addiert.

$addCipher = (c_1 \cdot c_2) \mod n^2$

In [12]:
# addition
def add_encrypted_numbers(c1, c2, n_square):
    return (c1 * c2) % n_square

# added cipher
added_cipher = add_encrypted_numbers(c1, c2, n**2)
print(added_cipher)

# added clear text
added_decrypted = decryption(added_cipher, lmbda, n, mu)
print(f"The sum of {num1} + {num2} = {added_decrypted}")

1432175365
The sum of 2 + 7 = 9


## Text Beispiel

In [13]:
def generate_keypair(bit_length):
    create_primes(bit_length)
    # create values
    n = (p * q)
    phi = (p - 1) * (q - 1)
    
    # check ggT
    result_ggT = ggT(n, phi)
    print("ggT = ", result_ggT)
    assert result_ggT == 1
    
    # choose random g
    group_g = multi_group_g(n)
    g = random.choice(group_g)
    print("Multiplikative Gruppe (Z/n^2Z)* für n =", n, "enthält", len(group_g), "Elemente.\nRandom int g: ", g)
    
    # final public Key
    pubKey = n, g

    lmbda = calc_lmbda(p, q)
    mu = calc_mu(g, lmbda, n)

    # final private Key
    privKey = lmbda, mu

    # choose random r
    r = choose_r(n)
    
    return pubKey, privKey, r

def encrypt_sentence(public_key, sentence):
    n, g = public_key
    ciphertext = []
    for letter in sentence:
        # convert letter in ASCII
        letter_num = ord(letter)
        # encrypt ASCII value
        encrypted_letter = encryption(g, letter_num, r, n)
        ciphertext.append(encrypted_letter)
    return ciphertext

def decrypt_sentence(ciphertext, private_key):
    lam, mu = private_key
    decrypted_sentence = ""
    for encrypted_letter in ciphertext:
        # decrypts ASCII value
        decrypted_letter_num = decryption(encrypted_letter, lmbda, n, mu)
        # ASCÌI to char
        decrypted_letter = chr(decrypted_letter_num)
        decrypted_sentence += decrypted_letter
    return decrypted_sentence

# Example
bit_length = 8
public_key, private_key, r = generate_keypair(bit_length)

sentence = "Hello, world!"
print("Original sentence:", sentence)

ciphertext = encrypt_sentence(public_key, sentence)
print("Encrypted sentence:", ciphertext)

decrypted_sentence = decrypt_sentence(ciphertext, private_key)
print("Decrypted sentence:", decrypted_sentence)

Bit length of 131 is: 8 bits
Bit length of 173 is: 8 bits
ggT =  1
Multiplikative Gruppe (Z/n^2Z)* für n = 49447 enthält 118901097 Elemente.
Random int g:  1013093713
Multiplikative Gruppe (Z/nZ)* für n = 49447 enthält 5078 Elemente.
Random int r:  5011
Original sentence: Hello, world!
Encrypted sentence: [1041618696, 907743447, 1771657709, 1771657709, 2105881072, 76382359, 1860370966, 2091906035, 2105881072, 190297417, 1771657709, 1909441000, 675861725]
Decrypted sentence: Hello, world!
